In [7]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import wfdb
from wfdb import processing

from gc import collect as collect_garbage
from psutil import virtual_memory
from os import scandir

In [8]:
%matplotlib widget

In [9]:
folder = "../../Deidentified-Raw-Waveforms/"
coldict = {
    "raw_waves_data_1a.csv": ["time", "257"], "raw_waves_data_1b.csv": ["time", "257", "258"], "raw_waves_data_1c.csv": ["time", "257", "258"], "raw_waves_data_1d.csv": ["time", "257", "258", "317"], 
    "raw_waves_data_1e.csv": ["time", "258"],

    "raw_waves_data_2a.csv": ["time", "257", "258"], "raw_waves_data_2b.csv": ["time", "258"], "raw_waves_data_2c.csv": ["time", "257"], "raw_waves_data_2d.csv": ["time", "257", "258"], 
    "raw_waves_data_2e.csv": ["time", "257", "258"],

    "raw_waves_data_3a.csv": ["time", "258"], "raw_waves_data_3b.csv": ["time", "258"], "raw_waves_data_3c.csv": ["time", "258"], "raw_waves_data_3d.csv": ["time", "258"], 
    "raw_waves_data_3e.csv": ["time", "257", "258", "317"],

    "raw_waves_data_4a.csv": ["time", "257", "258"], "raw_waves_data_4b.csv": ["time", "257", "258"], "raw_waves_data_4c.csv": ["time", "257"], "raw_waves_data_4d.csv": ["time", "257", "258"], 
    "raw_waves_data_4e.csv": ["time", "257", "258"],

    "raw_waves_data_5a.csv": ["time", "258"], "raw_waves_data_5b.csv": ["time", "258"], "raw_waves_data_5c.csv": ["time", "258"], "raw_waves_data_5d.csv": ["time", "258", "317"],
    "raw_waves_data_5e.csv": ["time", "258"],

    "raw_waves_data_6a.csv": ["time", "257", "258"], "raw_waves_data_6b.csv": ["time", "258"], "raw_waves_data_6c.csv": ["time", "258"], "raw_waves_data_6d.csv": ["time", "258"], "raw_waves_data_6e.csv": ["time", "258"],
    
    "raw_waves_data_7a.csv": ["time", "257", "258"], "raw_waves_data_7b.csv": ["time", "258"], "raw_waves_data_7c.csv": ["time", "258"], "raw_waves_data_7d.csv": ["time", "257", "258", "317"], 
    "raw_waves_data_7e.csv": ["time", "258"]
}

namedict = {
    "raw_waves_data_1a.csv": "1a", "raw_waves_data_1b.csv": "1b", "raw_waves_data_1c.csv": "1c", "raw_waves_data_1d.csv": "1d", "raw_waves_data_1e.csv": "1e",
    "raw_waves_data_2a.csv": "2a", "raw_waves_data_2b.csv": "2b", "raw_waves_data_2c.csv": "2c", "raw_waves_data_2d.csv": "2d", "raw_waves_data_2e.csv": "2e",
    "raw_waves_data_3a.csv": "3a", "raw_waves_data_3b.csv": "3b", "raw_waves_data_3c.csv": "3c", "raw_waves_data_3d.csv": "3d", "raw_waves_data_3e.csv": "3e",
    "raw_waves_data_4a.csv": "4a", "raw_waves_data_4b.csv": "4b", "raw_waves_data_4c.csv": "4c", "raw_waves_data_4d.csv": "4d", "raw_waves_data_4e.csv": "4e",
    "raw_waves_data_5a.csv": "5a", "raw_waves_data_5b.csv": "5b", "raw_waves_data_5c.csv": "5c", "raw_waves_data_5d.csv": "5d", "raw_waves_data_5e.csv": "5e",
    "raw_waves_data_6a.csv": "6a", "raw_waves_data_6b.csv": "6b", "raw_waves_data_6c.csv": "6c", "raw_waves_data_6d.csv": "6d", "raw_waves_data_6e.csv": "6e",
    "raw_waves_data_7a.csv": "7a", "raw_waves_data_7b.csv": "7b", "raw_waves_data_7c.csv": "7c", "raw_waves_data_7d.csv": "7d", "raw_waves_data_7e.csv": "7e"    
}

In [10]:
collect_garbage()
virtual_memory()

svmem(total=12655771648, available=6319271936, percent=50.1, used=6336499712, free=6319271936)

In [13]:
files = [file.name for file in scandir("../../Deidentified-Raw-Waveforms/")]
remove_files = ["raw_waves_data_7d.csv", "raw_waves_data_7e.csv"]
for i in range(20):
    files.pop(0)

for file in remove_files:
    files.remove(file)

In [14]:
files

['raw_waves_data_5a.csv',
 'raw_waves_data_5b.csv',
 'raw_waves_data_5c.csv',
 'raw_waves_data_5d.csv',
 'raw_waves_data_5e.csv',
 'raw_waves_data_6a.csv',
 'raw_waves_data_6b.csv',
 'raw_waves_data_6c.csv',
 'raw_waves_data_6d.csv',
 'raw_waves_data_6e.csv',
 'raw_waves_data_7a.csv',
 'raw_waves_data_7b.csv',
 'raw_waves_data_7c.csv']

In [15]:
# We will have an array telling us which of the files did not have their last
# chunk processed
for file in files:
    print("Starting now with file " + file)
    print(virtual_memory())

    # Get the columns
    cols = coldict[file]

    # Read in the data
    df = pd.read_csv(folder + file, usecols=coldict[file])
    print ("Data loaded in")

    # My little test of the files tells me the sampling frequency is 250 Hz
    freq = 250 

    # Create a complete signal in which to detect R-peaks
    signal = pd.Series(df[cols[1]]) # Column 0 is the time
    i=2
    while True:
        try:
            signal = signal.combine_first(df[cols[i]])
            i+=1
        except IndexError:
            break
    signal = signal.fillna(0)
    signal = pd.to_numeric(signal)
    print("Signals combined and filled in")

    # Delineate a chunk of length, say, 20000 that goes to the end of the signal, and find R-peaks inside
    try:
        hi = len(signal)
        lo = hi - 20000
        xqrs = processing.XQRS(sig=signal[lo:hi], fs=freq)
        xqrs.detect(verbose=False)
        rpeaks = list(lo + xqrs.qrs_inds)
    except IndexError:
        pass
    
    # Grab the time stamps
    df.loc[rpeaks, "time"].to_csv("final_chunks/missing_rpeaks_" + namedict[file] + ".csv")
    print("Output file written")

    # Delete all of the variables to save space
    del df
    del xqrs
    del signal
    del rpeaks
    collect_garbage()

    print("File " + namedict[file] + " processed")
    print(virtual_memory())


Starting now with file raw_waves_data_5a.csv
svmem(total=12655771648, available=6400323584, percent=49.4, used=6255448064, free=6400323584)
Data loaded in
Signals combined and filled in
Output file written
File 5a processed
svmem(total=12655771648, available=6479589376, percent=48.8, used=6176182272, free=6479589376)
Starting now with file raw_waves_data_5b.csv
svmem(total=12655771648, available=6479589376, percent=48.8, used=6176182272, free=6479589376)
Data loaded in
Signals combined and filled in
Output file written
File 5b processed
svmem(total=12655771648, available=6344142848, percent=49.9, used=6311628800, free=6344142848)
Starting now with file raw_waves_data_5c.csv
svmem(total=12655771648, available=6344142848, percent=49.9, used=6311628800, free=6344142848)
Data loaded in
Signals combined and filled in
Output file written
File 5c processed
svmem(total=12655771648, available=6827008000, percent=46.1, used=5828763648, free=6827008000)
Starting now with file raw_waves_data_5d.cs